In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
from create_matrix import *
import os
import sys
from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse_fraction
from common import compute_rmse

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}
APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
year = 2014
n_splits = 10
case=2
a=2
cost='abs'

In [3]:
def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

def get_tensor(df, dfc):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [5]:
region = "SanDiego"
sd_df, sd_dfc = create_matrix_single_region("SanDiego", year)
sd_tensor = get_tensor(sd_df, sd_dfc)
region = "Austin"
au_df, au_dfc = create_matrix_single_region("Austin", year)
au_tensor = get_tensor(au_df, au_dfc)

## Leave one cell out for transfer learning and normal learning

### algo: adagrad with static factors, no sparse constraints

In [66]:
import cPickle as pickle
def load_obj(name):
    with open(os.path.expanduser('~/git/' + name + '.pkl'), 'rb') as f:
        return pickle.load(f)

In [73]:
pred = load_obj("pred_month_None_gd_0")

In [87]:
pred['hvac']

{10: [                 0              1              2              3   \
  54    6.927621e-310  6.927621e-310  6.927621e-310  4.772960e+180   
  203   6.927621e-310  6.927621e-310  6.927621e-310  6.927621e-310   
  527   1.299638e+267  1.299638e+267  7.656861e+151  7.656861e+151   
  1450  8.660667e-154  3.503162e+151  8.802219e+199  8.648751e-154   
  
                   4              5              6              7   \
  54    6.927621e-310  1.644654e+161  1.016016e+160  6.927621e-310   
  203   6.927621e-310  6.927621e-310  6.927621e-310  6.927621e-310   
  527   7.656861e+151  4.822595e+151   4.662858e-66  7.502281e-120   
  1450  3.503162e+151  1.880039e+267  7.668369e+151   3.059634e-61   
  
                   8              9              10         11  
  54    6.927621e-310  6.927621e-310  1.302095e+190   6.964251  
  203   6.927621e-310  6.927621e-310   0.000000e+00  11.781544  
  527   8.653384e-154  3.127729e-120  3.105753e-120  27.979086  
  1450  7.889599e-120  8.65484

In [79]:
out_cv = {}
for random_seed in range(1):
    out_cv[random_seed] = {}
       
    for appliance in APPLIANCES_ORDER[1:]:
        out_cv[random_seed][appliance] = {}
        for f in range(10,20,10):
            s = pd.concat(pred[appliance][f]).ix[sd_df.index] 
            print s
            if appliance=="hvac":
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


                 0              1              2              3   \
54    6.927621e-310  6.927621e-310  6.927621e-310  4.772960e+180   
203   6.927621e-310  6.927621e-310  6.927621e-310  6.927621e-310   
527   1.299638e+267  1.299638e+267  7.656861e+151  7.656861e+151   
1450  8.660667e-154  3.503162e+151  8.802219e+199  8.648751e-154   
1524  -6.193755e-03  -1.130207e-02  -1.368340e-02  -6.193755e-03   
1731   1.363951e-03   1.282650e-03   6.037557e-03   1.363951e-03   
2031  -1.687137e-02  -3.528477e-02  -2.976022e-02  -1.687137e-02   
2354  -2.865270e-03  -5.352169e-03  -7.208659e-03  -2.865270e-03   
2606   2.471073e+02   8.998481e+01   3.666515e+01   1.163311e+02   
3687   7.282950e+01   9.117166e+01   3.169373e+03   1.752330e+02   
3864   1.143164e+03   1.988350e+03   1.238058e+03   2.104659e+01   
3938   3.278343e+02   4.325404e+02   4.815135e+02   4.936252e+02   
4083   4.385717e+02            NaN   3.199961e+02   4.468160e+02   
4095   8.730000e-02            NaN            Na

In [78]:
out_cv

{0: {'dw': {10: inf},
  'fridge': {10: inf},
  'hvac': {10: inf},
  'mw': {10: inf},
  'oven': {10: inf},
  'wm': {10: 1.8431356330984155e+150}}}

In [ ]:
import pickle
pred = {}
for static_fac in ['static', 'None']:
    for algo in ['adagrad']:
        for month in range(12):
            month = str(month)
            


for method in ['normal', 'transfer']:
    pred[method] = {}
    for algo in ['adagrad', 'gd', 'gd_decay']:
        pred[method][algo] = {}
        for cost in['abs', 'rel']:
            pred[method][algo][cost] = {}
            for a in [2, 3]:
                pred[method][algo][cost][a] = {}
                for lr in [0.01, 0.1, 1, 10, 100]:
                    lr = float(lr)
                    pred[method][algo][cost][a][lr] = {}
                    for random_seed in range(10):
                        print method, algo, cost, a, lr, random_seed
                        pred[method][algo][cost][a][lr][random_seed] = load_obj('pred/pred_' + method + '_' + algo + '_'
                                                                               + cost + '_' + str(a) + '_' + str(lr)
                                                                               + '_' + str(random_seed))

In [11]:
a = 2
b = 3
H_a, A_a, T_a = learn_HAT(case, au_tensor, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost ,T_known = np.ones(12).reshape(-1, 1)) 

In [13]:
m,n,o = sd_tensor.shape

In [33]:
pred_cell = np.empty([m,n,o])

In [34]:

for i in range(m):
    for j in range(1,n):
        for k in range(o):
            print i, j, k
            tensor_copy = sd_tensor.copy()
            tensor_copy[i, j, k] = np.NaN
            
            H, A, T = learn_HAT(case, tensor_copy, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost, A_known = A_a, T_known=np.ones(12).reshape(-1, 1))
            prediction = multiply_case(H, A, T, case)
            
            pred_cell[i][j][k] = prediction[i][j][k]

0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 11
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 2 10
0 2 11
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 3 10
0 3 11
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 4 10
0 4 11
0 5 0
0 5 1
0 5 2
0 5 3
0 5 4
0 5 5
0 5 6
0 5 7
0 5 8
0 5 9
0 5 10
0 5 11
0 6 0
0 6 1
0 6 2
0 6 3
0 6 4
0 6 5
0 6 6
0 6 7
0 6 8
0 6 9
0 6 10
0 6 11
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 1 10
1 1 11
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 2 10
1 2 11
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
1 3 6
1 3 7
1 3 8
1 3 9
1 3 10
1 3 11
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 4 8
1 4 9
1 4 10
1 4 11
1 5 0
1 5 1
1 5 2
1 5 3
1 5 4
1 5 5
1 5 6
1 5 7
1 5 8
1 5 9
1 5 10
1 5 11
1 6 0
1 6 1
1 6 2
1 6 3
1 6 4
1 6 5
1 6 6
1 6 7
1 6 8
1 6 9
1 6 10
1 6 11
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 1 8
2 1 9
2 1 10
2 1 11
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 2 5
2 

In [70]:
from scipy.optimize import nnls

n_splits = 10
case = 2
a = 2
b = 3
cost = 'abs'


pred_cv = {}
for random_seed in range(10):
    
    pred_cv[random_seed] = {}
    for appliance in APPLIANCES_ORDER:
        pred_cv[random_seed][appliance] = {f:[] for f in range(10, 110, 10)}

kf = KFold(n_splits=n_splits)
for random_seed in range(5):
    print "random seed: ", random_seed
    for train_percentage in range(10, 110, 10):
        print "training percentage: ", train_percentage
        rd = 0
        for train_max, test in kf.split(sd_df):
                print "round: ", rd

                num_train = int((train_percentage*len(train_max)/100)+0.5)
                num_test = len(test)

                # get the random training data from train_max based on then random seed
                if train_percentage==100:
                    train = train_max
                else:
                    train, _ = train_test_split(train_max, train_size = train_percentage/100.0, random_state=random_seed)

                # get the index of training and testing data
                train_ix = sd_df.index[train]
                test_ix = sd_df.index[test]
                print "test_ix: ", test_ix

                # create the tensor
                train_test_ix = np.concatenate([test_ix, train_ix])
                df_t, dfc_t = sd_df.ix[train_test_ix], sd_dfc.ix[train_test_ix]
                tensor = get_tensor(df_t, dfc_t)


                #############################################################################################
                # transfer learning: constant constraint, with A_known = A_a_const, with learn_HAT_constant
                ############################################################################################
                pred = np.empty([num_test, 7, 12])
                
                for i in range(1, 7):
                    for j in range(12):
                    
                        print i, j
                        tensor_copy = tensor.copy()
                        tensor_copy[:num_test, i, j] = np.NaN
                        H, A, T = learn_HAT(case, tensor_copy, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost, A_known=A_a,T_known = np.ones(12).reshape(-1, 1))
                        HAT = multiply_case(H, A, T, case)
                        pred[:, i, j] = HAT[:num_test, i, j]
                # get the prediction
                
                for appliance in APPLIANCES_ORDER:
                    pred_cv[random_seed][appliance][train_percentage].append(pd.DataFrame(pred[:, appliance_index[appliance], :], index=test_ix))

                rd += 1

random seed:  0
training percentage:  10
round:  0
test_ix:  Int64Index([54, 203, 527, 1450], dtype='int64')
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
round:  1
test_ix:  Int64Index([1524, 1731, 2031, 2354], dtype='int64')
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
round:  2
test_ix:  Int64Index([2606, 3687, 3864, 3938], dtype='int64')
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 

KeyboardInterrupt: 

In [83]:
pred_cv[0]['fridge']

{10: [             0          1          2          3          4          5   \
  54    15.067506  14.750169  15.152734  16.280202  20.258232  20.871764   
  203   20.193407  19.754779  20.324603  21.894663  27.649882  28.429111   
  527   43.488306  42.530516  43.775555  47.213606  60.046971  61.733262   
  1450  39.890370  39.044341  40.126148  43.140292  53.819649  55.435745   
  
               6          7          8          9          10         11  
  54    20.542532  24.713764  24.975883  20.594470  16.988741  21.117388  
  203   28.292457  34.113796  34.459890  28.016192  22.859454  28.580013  
  527   61.787706  74.596432  75.353648  60.804724  49.338222  61.865798  
  1450  54.638602  65.787352  66.417296  54.647597  45.000681  55.992838  ,
               0          1          2          3          4          5   \
  1524  43.674540  39.337543  40.519911  45.899222  58.411080  57.007935   
  1731  34.251842  30.978370  31.860719  35.910018  44.984866  44.152264   
  2031  1

In [54]:
pred_cv[0]['dw'][10]

[             0          1          2          3          4          5   \
 54     4.441050   4.374520   4.457078   4.687009   4.945526   5.192324   
 203    5.503513   5.430953   5.521882   5.773382   6.090897   6.353401   
 527   11.354342  11.216256  11.390482  11.870073  12.521796  13.012461   
 1450  11.623782  11.452640  11.665282  12.256978  12.932745  13.565594   
 
              6          7          8          9          10         11  
 54     4.288658   5.120256   5.142503   5.153532   4.816868   5.673344  
 203    5.416679   6.326773   6.352496   6.309071   5.915830   6.851801  
 527   11.295028  13.031156  13.082057  12.925400  12.142253  13.925949  
 1450  11.255776  13.395926  13.453595  13.465194  12.591277  14.795044  ,
              0          1          2          3          4          5   \
 1524  10.809674  10.263070  10.387690  11.045619  11.730628  12.102732   
 1731   9.438079   8.860635   8.989631   9.682273  10.311600  10.761934   
 2031   4.138151   3.872244

In [38]:
pred_cell[:, 1, :]

array([[   0.        ,    0.        ,    0.        ,   11.18723443,
          85.89969937,   54.95676031,  138.22499142,  165.33004297,
         174.06390724,   64.82408541,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   16.36920788,
         122.07919527,   79.19176856,  196.30009438,  243.430618  ,
         282.42734073,   99.45670315,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   39.09422878,
         291.55912891,  194.69024705,  496.86355717,  573.53475427,
         598.77516629,  228.69533   ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   34.03037097,
         227.21695396,  149.7512468 ,  391.03006304,  424.19545117,
         456.80554756,  163.1795575 ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   40.52508894,
         302.23027798,  201.81596682,  515.04890814,  622.49467606,
         683.39460301,  253.12000959,    0. 

In [41]:
out = {}
for appliance in APPLIANCES_ORDER[1:]:
    out[appliance] = {}
    
    s = pd.DataFrame(pred_cell[:, appliance_index[appliance], :], index = sd_df.index)
    
    if appliance=="hvac":
        out[appliance] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
    else:   
        out[appliance] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

In [42]:
out

{'dw': 1.8419311486511902,
 'fridge': 9.8563926947604124,
 'hvac': 13.355191358765531,
 'mw': 2.0429952976254757,
 'oven': 2.0290824167898225,
 'wm': 0.80628242095805347}

In [85]:
out_cv = {}
for random_seed in range(1):
    out_cv[random_seed] = {}
       
    for appliance in APPLIANCES_ORDER[1:]:
        out_cv[random_seed][appliance] = {}
        for f in range(10,20,10):
            s = pd.concat(pred_cv[random_seed][appliance][f]).ix[sd_df.index]
            if appliance=="hvac":
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [86]:
out_cv

{0: {'dw': {10: 1.8511217053352107},
  'fridge': {10: 10.409705903351151},
  'hvac': {10: 15.199581098682868},
  'mw': {10: 1.9182482790934556},
  'oven': {10: 2.0871632432596376},
  'wm': {10: 0.81793246666963448}}}